In [ ]:
#######################################################################################################################
# Project: Deep Virtual Rapport Agent (rapport model)
#
#     Jan Ondras (jo951030@gmail.com)
#     Institute for Creative Technologies, University of Southern California
#     April-October 2019
#
#######################################################################################################################
# Plot the outputs from the Google ASR speech-to-text: binary voice activity and audio waveform for each subject.
#     Saved as png plots.
#######################################################################################################################

In [1]:
# Check the counts of speaker tags for each audio

import json
import glob
import numpy as np
import pandas as pd
import soundfile as sf
from matplotlib import pyplot as plt
from collections import Counter

# Path to STT outputs
stt_outputs_dir = f'/home/ICT2000/jondras/dvra_datasets/mimicry/voice_activity_detection/speech_to_text_google/dumps'

# Iterate over csv output files
for i, stt_output_file in enumerate(sorted(glob.glob(f'{stt_outputs_dir}/*_G_ASR_dump.json'))):
    
    output_filename_split = stt_output_file.split('/')[-1][:-4].split('_')
    audio_basename = '_'.join(output_filename_split[:5])
    print(audio_basename)
    
    # Load json dump outputs from STT
    with open(stt_output_file, "r") as json_file:
        data = json.load(json_file)
#         print(data)
        
        tags = []
        # Check if there are speaker tags and what they are        
        for result in data['results']:
            for word in result['alternatives'][0]['words']:
                if 'speakerTag' in word.keys():
                    tags.append( word['speakerTag'] )
#         data[]
    print(f'\tTags: {Counter(tags)} \n\t# results: {len(data["results"])}\n')

print(f'Processed {i + 1} Google speech2text output files.')

sessid_01_P1_sid_09
	Tags: Counter({1: 642, 2: 67}) 
	# results: 19

sessid_01_P2_sid_02
	Tags: Counter({2: 602, 1: 110}) 
	# results: 23

sessid_02_P1_sid_09
	Tags: Counter({2: 1102, 1: 246}) 
	# results: 44

sessid_02_P2_sid_17
	Tags: Counter({1: 980, 2: 266}) 
	# results: 35

sessid_03_P1_sid_17
	Tags: Counter() 
	# results: 9

sessid_03_P2_sid_02
	Tags: Counter({2: 347, 1: 101}) 
	# results: 22

sessid_04_P1_sid_12
	Tags: Counter({1: 548, 2: 151}) 
	# results: 35

sessid_04_P2_sid_23
	Tags: Counter() 
	# results: 21

sessid_05_P1_sid_12
	Tags: Counter({2: 1064}) 
	# results: 20

sessid_05_P2_sid_21
	Tags: Counter({1: 1818, 2: 1}) 
	# results: 19

sessid_06_P1_sid_23
	Tags: Counter({2: 588, 1: 256}) 
	# results: 31

sessid_06_P2_sid_21
	Tags: Counter({2: 1186, 1: 789}) 
	# results: 27

sessid_07_P1_sid_09
	Tags: Counter({2: 976, 1: 264}) 
	# results: 27

sessid_07_P2_sid_01
	Tags: Counter({1: 1192, 2: 307}) 
	# results: 36

sessid_08_P1_sid_09
	Tags: Counter({1: 978, 2: 208}) 
	# re

In [ ]:
# both 1 and 2
#     84
# 1 only
#      0
# 2 only
#      4
# none
#     18

In [1]:
# Plot the outputs from the Google Speech-2-Text

import json
import glob
import numpy as np
import pandas as pd
import soundfile as sf
from matplotlib import pyplot as plt

# Path to STT outputs
stt_outputs_dir = f'/home/ICT2000/jondras/dvra_datasets/mimicry/voice_activity_detection/speech_to_text_google/dumps'
mono_audio_dir = '/home/ICT2000/jondras/dvra_datasets/mimicry/audio/audio_separated_16kHz_flac/'

# Iterate over csv output files and audio files
for i, stt_output_file in enumerate(sorted(glob.glob(f'{stt_outputs_dir}/*_G_ASR_dump.json'))):
        
    output_filename_split = stt_output_file.split('/')[-1][:-4].split('_')
    audio_basename = '_'.join(output_filename_split[:5])
    print(audio_basename)
#     if int(audio_basename[7:9]) < 54:
#         continue

    # Load audio signal
    audio_signal, samplerate = sf.read(f'{mono_audio_dir}/{audio_basename}.flac')
    print(f'\tSampling rate: {samplerate}')
    x_audio_signal = np.arange(0, len(audio_signal) / samplerate, 1. / samplerate)[:len(audio_signal)]
    
    # Load json dump outputs from STT and generate binary voice activity for each speaker
    # Speaker tags are 1 and 2 only
    speaker_tags = [1, 2]
    bin_voice_activity = dict()
    for speaker_tag in speaker_tags:
        bin_voice_activity[speaker_tag] = np.zeros(len(audio_signal), dtype=int)
        
    with open(stt_output_file, "r") as json_file:
        data = json.load(json_file)
#         print(data)

        # Check if speaker tags were generated (if yes, take words from the last result only)
        if 'speakerTag' in data['results'][-1]['alternatives'][0]['words'][0].keys():
            for word in data['results'][-1]['alternatives'][0]['words']:
                # Skip "s" at the end of the string
                start_time = float(word['startTime'][:-1])
                end_time = float(word['endTime'][:-1])
                bin_voice_activity[word['speakerTag']] = np.where(
                    (x_audio_signal >= start_time) & (x_audio_signal <= end_time), 
                    1, bin_voice_activity[word['speakerTag']])

        # If no speaker tags were assigned, take words from all results and assign speaker tag 1
        else:
            speaker_tag = 1
            for result in data['results']:
                for word in result['alternatives'][0]['words']:
                    # Skip "s" at the end of the string
                    start_time = float(word['startTime'][:-1])
                    end_time = float(word['endTime'][:-1])
                    bin_voice_activity[speaker_tag] = np.where(
                        (x_audio_signal >= start_time) & (x_audio_signal <= end_time), 
                        1, bin_voice_activity[speaker_tag])
                    
    # Plot raw audio signal and bin_voice_activity
#     plt.figure(figsize=[15,6])
    plt.figure(figsize=[25,6])
    plt.title(audio_basename)
    plt.plot(x_audio_signal, audio_signal, 'g-', alpha=0.3, label='audio signal')#,linewidth=2.0)
    for speaker_tag in speaker_tags:
        plt.plot(x_audio_signal, bin_voice_activity[speaker_tag] - speaker_tag*1.5 + 1.5, label=f'bin_voice_activity_{speaker_tag}')#,linewidth=2.0)
#     plt.xlim(110, 135)
    plt.ylim(-1.6, 1.6)
    plt.xlabel('Time (s)')
    plt.legend()
    plt.savefig(f'./speaker_diarization_plots_google/{audio_basename}.png')
#     plt.show()
    
#     break
        
print(f'Processed {i + 1} Google speech2text output files.')

sessid_01_P1_sid_09
sessid_01_P2_sid_02
sessid_02_P1_sid_09
sessid_02_P2_sid_17
sessid_03_P1_sid_17
sessid_03_P2_sid_02
sessid_04_P1_sid_12
sessid_04_P2_sid_23
sessid_05_P1_sid_12
sessid_05_P2_sid_21
sessid_06_P1_sid_23
sessid_06_P2_sid_21
sessid_07_P1_sid_09
sessid_07_P2_sid_01
sessid_08_P1_sid_09
sessid_08_P2_sid_04
sessid_09_P1_sid_01
sessid_09_P2_sid_04
sessid_10_P1_sid_09
sessid_10_P2_sid_34
sessid_11_P1_sid_09
sessid_11_P2_sid_15
sessid_12_P1_sid_15
sessid_12_P2_sid_11
sessid_13_P1_sid_09
sessid_13_P2_sid_19
sessid_14_P1_sid_19
sessid_14_P2_sid_06
sessid_15_P1_sid_09
sessid_15_P2_sid_16
sessid_16_P1_sid_24
sessid_16_P2_sid_16
sessid_17_P1_sid_09
sessid_17_P2_sid_43
sessid_18_P1_sid_03
sessid_18_P2_sid_43
sessid_19_P1_sid_09
sessid_19_P2_sid_22
sessid_20_P1_sid_09
sessid_20_P2_sid_50
sessid_21_P1_sid_50
sessid_21_P2_sid_22
sessid_22_P1_sid_12
sessid_22_P2_sid_18
sessid_23_P1_sid_12
sessid_23_P2_sid_39
sessid_24_P1_sid_18
sessid_24_P2_sid_39
sessid_25_P1_sid_12
sessid_25_P2_sid_27
